In [26]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

# Qiskit Imports (Quantum Optimization)
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer import Aer
from qiskit_aer.primitives import Sampler  

# Step 1: Download Historical Stock Data
stocks = ["TCS.NS", "INFY.NS", "HDFCBANK.NS", "RELIANCE.NS", "ITC.NS", "BAJFINANCE.NS"]
df = yf.download(stocks, start="2021-01-01", end="2024-01-01")['Close']

# Step 2: Calculate Returns & Covariance Matrix
returns = df.pct_change().dropna()
expected_returns = returns.mean()
cov_matrix = returns.cov()

# Step 3: Construct QUBO Matrix
assets = len(expected_returns)
Q = np.zeros((assets, assets))
penalty = 1 / np.mean(abs(cov_matrix.values))  # Adjust for risk-return balance

for i in range(assets):
    Q[i, i] = -expected_returns.iloc[i] + penalty * cov_matrix.iloc[i, i]
    for j in range(i + 1, assets):
        Q[i, j] = penalty * cov_matrix.iloc[i, j]
        Q[j, i] = Q[i, j]

# Step 4: Classical Optimization (Scipy)
def classical_solver(Q):
    def objective(x):
        return sum(Q[i, j] * x[i] * x[j] for i in range(assets) for j in range(assets))
    
    x0 = np.random.randint(0, 2, assets)  # Initial random binary solution
    bounds = [(0, 1) for _ in range(assets)]  # Binary constraint
    result = minimize(objective, x0, bounds=bounds, method='COBYLA')
    
    # Convert floating values to binary (0 or 1)
    x_opt = np.round(result.x).astype(int)
    return x_opt, result.fun

classical_solution, classical_value = classical_solver(Q)

# Step 5: Quantum Optimization (QAOA on IBMQ)
# Define QUBO problem for Qiskit
qubo = QuadraticProgram()
for i in range(assets):
    qubo.binary_var(f"x{i}")

linear_terms = {f"x{i}": Q[i, i] for i in range(assets)}
quadratic_terms = {(f"x{i}", f"x{j}"): Q[i, j] for i in range(assets) for j in range(i + 1, assets)}
qubo.minimize(linear=linear_terms, quadratic=quadratic_terms)


[*********************100%***********************]  6 of 6 completed


In [27]:
# Load IBMQ backend
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2, Session, Options, Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import QAOAAnsatz


####### Real Backend

# QiskitRuntimeService.save_account(
#             channel="ibm_quantum",
#             token="ef7cd82468de50529639b7e506dab7dbb30cc6ed636948a42181ca9ef65744e57e24c53f0d366eef7aec01a9f34819e79b777a7e6beb0fc618f9f5a92976922d",
#             overwrite=True)

# service = QiskitRuntimeService()
# backend = service.backend("ibm_sherbrooke")

# qubit_op = SparsePauliOp.from_list([("ZZII", 1), ("IZZI", 1), ("IIZZ", 1), ("ZIIZ", 1)])

# ansatz = QAOAAnsatz(qubit_op, reps=10)
# pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
# transpiled_ansatz = pm.run(ansatz)

# options = Options(
#     # Execution settings
#     default_shots=1000,  # Direct shot specification
    
#     # Resilience settings (error mitigation)
#     resilience= {
#         "level": 1  # Nested under 'resilience' key
#     }
# )

# with Session(backend=backend) as session:  
#     sampler = SamplerV2(options=options)

    # qaoa = QAOA(sampler=sampler, optimizer=COBYLA(), reps=10)
    # meo = MinimumEigenOptimizer(qaoa)
    # qaoa_result = meo.solve(qubo)

    # qaoa_solution = np.array([int(qaoa_result.x[i]) for i in range(assets)])
    # qaoa_value = qaoa_result.fval


# backend = provider.get_backend("ibmq_qasm_simulator")  # Change to real backend like 'ibmq_mumbai' if available
# sampler = Aer.get_backend('aer_simulator')  # Local simulator for testing

In [33]:

#######  Fake Backend

from qiskit_ibm_runtime.fake_provider import FakeMumbaiV2  
from qiskit_ibm_runtime.options import SamplerOptions

fake_backend = FakeMumbaiV2()

# pm = generate_preset_pass_manager(
#     backend=fake_backend,
#     optimization_level=1,  
#     basis_gates=["rz", "sx", "cx", "x"]  
# )

# num_qubits = 27  
# mixer_operator = SparsePauliOp("X" * num_qubits)

# qubit_op = SparsePauliOp.from_list([("ZZII", 1), ("IZZI", 1), ("IIZZ", 1), ("ZIIZ", 1)])
# qubit_op = SparsePauliOp.from_list([
#     ("Z" + "I" * 26, 1),               # Z on qubit 0
#     ("ZZ" + "I" * 25, 1),              # ZZ on qubits 0,1
#     ("IZZI" + "I" * 23, 1),            # ZZ on qubits 1,2
#     ("IIZZ" + "I" * 23, 1),            # ZZ on qubits 2,3
#     ("ZIIZ" + "I" * 23, 1)             # Z on qubit 0 and 3
# ])
# ansatz = QAOAAnsatz(qubit_op, reps=10,mixer_operator=mixer_operator)

# print(f"Original parameters: {ansatz.num_parameters}")  
# transpiled_ansatz = pm.run(ansatz)
# print(f"Transpiled parameters: {transpiled_ansatz.num_parameters}") 

# from qiskit import transpile 
# transpiled_ansatz = transpile(
#     ansatz,
#     backend=fake_backend,
#     basis_gates=["rz", "sx", "cx", "x"],
#     optimization_level=1,
#     layout_method="trivial",  # Avoid qubit remapping issues
# )

# sampler = Sampler(
#     mode=fake_backend,
#     options=SamplerOptions(
#             default_shots=1024,
#             execution={"init_qubits": True}
#     )
# )

# from qiskit_aer.primitives import Sampler  # Use Aer's V1 Sampler

# # Fake backend setup (for transpilation only)
# from qiskit_ibm_runtime.fake_provider import FakeMumbaiV2  
# fake_backend = FakeMumbaiV2()

# # Transpile ansatz for the fake backend
# pm = generate_preset_pass_manager(backend=fake_backend, optimization_level=1)
# ansatz = QAOAAnsatz(qubit_op, reps=10)
# transpiled_ansatz = pm.run(ansatz)

# # Use Aer's Sampler (V1 compatible)
# sampler = Sampler()  # Creates a local simulator-based sampler

# # Configure QAOA with correct parameters
# qaoa = QAOA(
#     sampler=sampler,
#     optimizer=COBYLA(maxiter=100),
#     reps=10,
#     initial_point=np.random.rand(20)  # Match number of parameters
# )

# from sampleradapter import SamplerV2Adapter
# sampler = SamplerV2Adapter(sampler)

# qaoa = QAOA(sampler=sampler, optimizer=COBYLA(), reps=10)
# qaoa.ansatz = transpiled_ansatz
# qaoa.initial_point = [0.1] * transpiled_ansatz.num_parameters 

# print(transpiled_ansatz.draw())

num_assets = len(expected_returns)
print(num_assets)

qubit_op = SparsePauliOp.from_list([
    ("ZZIIII", 1), ("ZIZIII", 1), ("ZIIIZI", 1),  # Example interactions
    ("IZZIII", 1), ("IIZZII", 1), ("IIIZIZ", 1)   # Adjust according to your problem
])


# Create QAOA ansatz with proper mixer operator
mixer_op = SparsePauliOp("X" * num_assets)  # Match number of assets/qubits
ansatz = QAOAAnsatz(qubit_op, reps=1,mixer_operator=mixer_op)

pm = generate_preset_pass_manager(
    backend=fake_backend,
    optimization_level=0,  
    # layout_method="trivial",
    # routing_method="none"
)
transpiled_circuit = pm.run(ansatz)
transpiled_circuit = transpiled_circuit.decompose().decompose()

print(f"Original parameters: {ansatz.num_parameters}") 
print(f"Transpiled parameters: {transpiled_circuit.num_parameters}") 

options=SamplerOptions(
            default_shots=1024,
            execution={"init_qubits": True}
    )

sampler = SamplerV2(mode=fake_backend, options=options)
# Configure QAOA with transpiled circuit
qaoa = QAOA(
    sampler=sampler,
    optimizer=COBYLA(maxiter=20),
    initial_point=np.random.rand(transpiled_circuit.num_parameters),
)
qaoa.ansatz = transpiled_circuit
meo = MinimumEigenOptimizer(qaoa)
qaoa_result = meo.solve(qubo)


qaoa_solution = np.array([int(qaoa_result.x[i]) for i in range(assets)])
qaoa_value = qaoa_result.fval


6
Original parameters: 2
Transpiled parameters: 2


TypeError: SamplerV2.run() takes 2 positional arguments but 3 were given

In [ ]:
from qiskit import transpile
from qiskit_aer import Aer
from qiskit_algorithms import 
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA

# 1. Configure quantum instance with fake backend
quantum_instance = QuantumInstance(
    backend=FakeMumbaiV2(),
    shots=1024,
    optimization_level=0,  # Critical for parameter preservation
    initial_layout=list(range(num_assets))  # Force trivial layout
)
# 2. Manual transpilation with parameter preservation
ansatz = QAOAAnsatz(qubit_op, reps=2, mixer_operator=mixer_op)
transpiled_circuit = transpile(
    ansatz,
    backend=quantum_instance.backend,
    optimization_level=0,
    layout_method="trivial",
    routing_method="none"
)

# 3. Verify parameter count
print(f"Parameters: {transpiled_circuit.num_parameters}")  # Should be 4 (2 reps)

# 4. Configure QAOA with manual execution
qaoa = QAOA(
    quantum_instance=quantum_instance,
    optimizer=COBYLA(maxiter=50),
    initial_point=np.random.rand(transpiled_circuit.num_parameters),
    callback=lambda count, params, value, meta: print(f"Iteration {count}: {value}")
)

# 5. Create manual executor function
def execute_circuit(params):
    # Bind parameters and transpile
    bound_circuit = transpiled_circuit.assign_parameters(params)
    tqc = transpile(bound_circuit, quantum_instance.backend)
    
    # Execute job
    job = quantum_instance.backend.run(tqc, shots=1024)
    counts = job.result().get_counts()
    
    # Calculate expectation value
    return np.sum([v * (counts[k] if k in counts else 0) 
                  for k, v in qubit_op.to_matrix().diagonal()])

# 6. Manual optimization loop
optimizer = COBYLA(maxiter=50)
result = optimizer.minimize(
    fun=lambda params: execute_circuit(params),
    x0=qaoa.initial_point
)

# 7. Process results
qaoa_result = MinimumEigenOptimizerResult()
qaoa_result.x = result.x
qaoa_result.fval = result.fun

ImportError: cannot import name 'QuantumInstance' from 'qiskit_algorithms' (C:\Users\kumar\AppData\Roaming\Python\Python312\site-packages\qiskit_algorithms\__init__.py)

In [ ]:
# Step 6: Print Results
print("\n===== Portfolio Optimization Results =====")
print("Classical Optimization:")
print("Selected Stocks:", [stocks[i] for i in range(assets) if classical_solution[i] == 1])
print("Objective Value:", classical_value)

print("\nQAOA Quantum Optimization (IBMQ):")
print("Selected Stocks:", [stocks[i] for i in range(assets) if qaoa_solution[i] == 1])
print("Objective Value:", qaoa_value)

# Step 7: Visualization
plt.figure(figsize=(10, 5))
methods = ["Classical", "QAOA (IBMQ)"]
values = [classical_value, qaoa_value]

sns.barplot(x=methods, y=values, palette=["blue", "green"])
plt.title("Portfolio Optimization: Classical vs Quantum")
plt.ylabel("Objective Function Value (Lower is Better)")
plt.show()